In [27]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10
import tensorflow as tf
# Network Parameters
num_hidden_1 = 256 # 1st layer num features
num_hidden_2 = 128
num_hidden_3 = 64
num_hidden_4 = 32# 2nd layer num features (the latent dim)
num_input = 27 
num_classes = 7

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'encoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_3])),
    'encoder_h4': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_4])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_4, num_hidden_3])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_2])),
    'decoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h4': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_4, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([num_hidden_3])),
    'encoder_b4': tf.Variable(tf.random_normal([num_hidden_4])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_3])),
    'decoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b3': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b4': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [28]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                                   biases['encoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['encoder_h4']),
                                   biases['encoder_b4']))
    
    return layer_4


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                   biases['decoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
                                   biases['decoder_b4']))
    return layer_4

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [29]:
from numpy import genfromtxt
my_data = genfromtxt('./DataForBiplab/2017Kharif_NDVI1.csv', delimiter=',')
my_data1 = genfromtxt('./DataForBiplab/2017Kharif_VV1.csv', delimiter=',')
my_data0 = genfromtxt('./DataForBiplab/2017Kharif_VH1.csv', delimiter=',')

In [30]:
data = my_data[1:,2:]
data1 = my_data1[1:,2:]
data0 = my_data0[1:,2:]
print(data.shape)
print(data1.shape)
print(data0.shape)

(546, 13)
(546, 14)
(546, 14)


In [31]:
print(data.max())
print(data1.max())
print(data0.max())

0.855952701
3.165250778
0.209239975


In [32]:
import numpy as np
data = np.array(data)
data1 = np.array(data1)
data0 = np.array(data0)
print(data.shape)
print(data1.shape)
print(data0.shape)

(546, 13)
(546, 14)
(546, 14)


In [33]:
import pandas
colnames = ['Latitude', 'Longitude', 'CropType','SampleYear','ShowingPeriod'
           , 'HarvestingPeriod', 'Others']
data3 = pandas.read_csv('./DataForBiplab/FieldDataKharif2017.csv', names=colnames)

In [34]:
ctype = data3.CropType.tolist()
print(ctype)

['CropType', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcan

In [35]:
p = np.array(ctype)
print(p.shape)

(547,)


In [36]:
lbl1 = ctype
label = []
for i in range(1,547):
    if(lbl1[i] == "Paddy"):
        label += [0]
    if(lbl1[i] == "Sugarcane"):
        label += [1]
    if(lbl1[i] == "PearlMillet"):
        label += [2]
    if(lbl1[i] == "PigeonPea"):
        label += [3]
    if(lbl1[i] == "Sorghum"):
        label += [4]
    if(lbl1[i] == "Forest/NonCropland"):
        label += [5]
    if(lbl1[i] == "Habitation"):
        label += [6]
    
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 

In [37]:
label = np.array(label)
print(label.shape)

(546,)


In [38]:
print(data.shape)
print(data1.shape)
print(data0.shape)
data1=np.log10(data1)
data0=np.log10(data0)
data4=data1/data0
data2 = np.concatenate((data,data4), axis=1)
print(data2.shape)

(546, 13)
(546, 14)
(546, 14)
(546, 27)


In [39]:
x1=[]
x2=[]
x3=[]
x4=[]
x5=[]
x6=[]
x7=[]
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
l6=[]
l7=[]
for i in range(0, label.shape[0]):
    if(label[i]==0):
        x1.append(data2[i])
        l1.append([1,0,0,0,0,0,0])
    if(label[i]==1):
        x2.append(data2[i])
        l2.append([0,1,0,0,0,0,0])
    if(label[i]==2):
        x3.append(data2[i])
        l3.append([0,0,1,0,0,0,0])
    if(label[i]==3):
        x4.append(data2[i])
        l4.append([0,0,0,1,0,0,0])
    if(label[i]==4):
        x5.append(data2[i])
        l5.append([0,0,0,0,1,0,0])
    if(label[i]==5):
        x6.append(data2[i])
        l6.append([0,0,0,0,0,1,0])
    if(label[i]==6):
        x7.append(data2[i])
        l7.append([0,0,0,0,0,0,1])
x1=np.array(x1)
x2=np.array(x2)
x3=np.array(x3)
x4=np.array(x4)
x5=np.array(x5)
x6=np.array(x6)
x7=np.array(x7)

l1=np.array(l1)
l2=np.array(l2)
l3=np.array(l3)
l4=np.array(l4)
l5=np.array(l5)
l6=np.array(l6)
l7=np.array(l7)

print(l7.shape)
print(x6.shape)

(209, 7)
(34, 27)


In [40]:
from sklearn.utils import shuffle
data2, label = shuffle(data2, label, random_state = 0)

In [15]:
# 80% train 20% test (NDVIclean)
X_train = data2[0:436,:]
X_test = data2[436:,:]
y_train = label[0:436]
y_test = label[436:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(436, 27)
(110, 27)
(436,)
(110,)


In [16]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(436, 7)
(110, 7)


/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.73698624
Epoch: 2 cost = 0.67986703
Epoch: 3 cost = 0.64542069
Epoch: 4 cost = 0.61690579
Epoch: 5 cost = 0.59418267
Epoch: 6 cost = 0.57507501
Epoch: 7 cost = 0.55702001
Epoch: 8 cost = 0.53927628
Epoch: 9 cost = 0.52236899
Epoch: 10 cost = 0.50683002
Epoch: 11 cost = 0.49246699
Epoch: 12 cost = 0.47862283
Epoch: 13 cost = 0.46487780
Epoch: 14 cost = 0.45142136
Epoch: 15 cost = 0.43890258
Epoch: 16 cost = 0.42787701
Epoch: 17 cost = 0.41845288
Epoch: 18 cost = 0.41047915
Epoch: 19 cost = 0.40376804
Epoch: 20 cost = 0.39815801
Epoch: 21 cost = 0.39354177
Epoch: 22 cost = 0.38986301
Epoch: 23 cost = 0.38680667
Epoch: 24 cost = 0.38333994
Epoch: 25 cost = 0.37849562
Epoch: 26 cost = 0.37260439
Epoch: 27 cost = 0.36651759
Epoch: 28 cost = 0.36001136
Epoch: 29 cost = 0.35385656
Epoch: 30 cost = 0.35255228
Epoch: 31 cost = 0.35146147
Epoch: 32 cost = 0.33540413
Epoch: 33 cost = 0.30404292
Epoch: 34 cost = 0.26263852
Epoch: 35 cost = 0.31507874
Epoch: 36 cost = 0.33565839
E

Epoch: 293 cost = 0.01617010
Epoch: 294 cost = 0.01603123
Epoch: 295 cost = 0.01653564
Epoch: 296 cost = 0.01585081
Epoch: 297 cost = 0.01564794
Epoch: 298 cost = 0.01617938
Epoch: 299 cost = 0.01618622
Epoch: 300 cost = 0.01624373
Epoch: 301 cost = 0.41562862
Epoch: 302 cost = 1.08021096
Epoch: 303 cost = 1.00388273
Epoch: 304 cost = 0.16861265
Epoch: 305 cost = 0.11534007
Epoch: 306 cost = 0.07120309
Epoch: 307 cost = 0.05269863
Epoch: 308 cost = 0.03625831
Epoch: 309 cost = 0.02579034
Epoch: 310 cost = 0.02182510
Epoch: 311 cost = 0.02080224
Epoch: 312 cost = 0.01938256
Epoch: 313 cost = 0.01882406
Epoch: 314 cost = 0.01894028
Epoch: 315 cost = 0.01782963
Epoch: 316 cost = 0.01796019
Epoch: 317 cost = 0.01800036
Epoch: 318 cost = 0.01734054
Epoch: 319 cost = 0.01731303
Epoch: 320 cost = 0.01737915
Epoch: 321 cost = 0.01679327
Epoch: 322 cost = 0.01657760
Epoch: 323 cost = 0.01673470
Epoch: 324 cost = 0.01599849
Epoch: 325 cost = 0.01689179
Epoch: 326 cost = 0.01619520
Epoch: 327 cos

Epoch: 579 cost = 0.39162524
Epoch: 580 cost = 0.05753712
Epoch: 581 cost = 0.01572300
Epoch: 582 cost = 0.01609165
Epoch: 583 cost = 0.01592416
Epoch: 584 cost = 0.01606315
Epoch: 585 cost = 0.01511904
Epoch: 586 cost = 0.01585082
Epoch: 587 cost = 0.01573556
Epoch: 588 cost = 0.01622549
Epoch: 589 cost = 0.01615641
Epoch: 590 cost = 0.01491273
Epoch: 591 cost = 0.01526248
Epoch: 592 cost = 0.01492928
Epoch: 593 cost = 0.01530527
Epoch: 594 cost = 0.01543465
Epoch: 595 cost = 0.01632018
Epoch: 596 cost = 0.01523260
Epoch: 597 cost = 0.01561063
Epoch: 598 cost = 0.01556632
Epoch: 599 cost = 0.01492653
Epoch: 600 cost = 0.01574557
Epoch: 601 cost = 0.01615107
Epoch: 602 cost = 0.01596474
Epoch: 603 cost = 0.01466887
Epoch: 604 cost = 0.01525236
Epoch: 605 cost = 0.01546824
Epoch: 606 cost = 0.01559888
Epoch: 607 cost = 0.01511551
Epoch: 608 cost = 0.01528089
Epoch: 609 cost = 0.01513819
Epoch: 610 cost = 0.01432838
Epoch: 611 cost = 0.01571438
Epoch: 612 cost = 0.01534986
Epoch: 613 cos

Epoch: 870 cost = 0.01481057
Epoch: 871 cost = 0.01498031
Epoch: 872 cost = 0.01527926
Epoch: 873 cost = 0.01552417
Epoch: 874 cost = 0.01552008
Epoch: 875 cost = 0.01525823
Epoch: 876 cost = 0.01473957
Epoch: 877 cost = 0.01470115
Epoch: 878 cost = 0.01526355
Epoch: 879 cost = 0.01446119
Epoch: 880 cost = 0.01543938
Epoch: 881 cost = 0.01591542
Epoch: 882 cost = 0.01462570
Epoch: 883 cost = 0.01454904
Epoch: 884 cost = 0.01514988
Epoch: 885 cost = 0.01537705
Epoch: 886 cost = 0.01652706
Epoch: 887 cost = 0.01503765
Epoch: 888 cost = 0.01571201
Epoch: 889 cost = 0.01585899
Epoch: 890 cost = 0.01553659
Epoch: 891 cost = 0.01494491
Epoch: 892 cost = 0.01454460
Epoch: 893 cost = 0.01472901
Epoch: 894 cost = 0.01459378
Epoch: 895 cost = 0.01558186
Epoch: 896 cost = 0.01479691
Epoch: 897 cost = 0.01426479
Epoch: 898 cost = 0.01465706
Epoch: 899 cost = 0.01448728
Epoch: 900 cost = 0.01565017
Epoch: 901 cost = 0.01533623
Epoch: 902 cost = 0.01619192
Epoch: 903 cost = 0.01547155
Epoch: 904 cos

In [41]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
print(y_test.shape)

(110, 7)


In [20]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.79090905


In [21]:
res1=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x1, Y: l1}))
res2=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x2, Y: l2}))
res3=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x3, Y: l3}))
res4=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x4, Y: l4}))
res5=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x5, Y: l5}))
res6=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x6, Y: l6}))
res7=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x7, Y: l7}))
print(res6)

[5 5 5 5 1 1 1 5 5 1 1 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 5 5 5 5]


In [22]:
confusion=[]
x1=np.sum(np.equal(res1, 0))
x2=np.sum(np.equal(res1, 1))
x3=np.sum(np.equal(res1, 2))
x4=np.sum(np.equal(res1, 3))
x5=np.sum(np.equal(res1, 4))
x6=np.sum(np.equal(res1, 5))
x7=np.sum(np.equal(res1, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res2, 0))
x2=np.sum(np.equal(res2, 1))
x3=np.sum(np.equal(res2, 2))
x4=np.sum(np.equal(res2, 3))
x5=np.sum(np.equal(res2, 4))
x6=np.sum(np.equal(res2, 5))
x7=np.sum(np.equal(res2, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res3, 0))
x2=np.sum(np.equal(res3, 1))
x3=np.sum(np.equal(res3, 2))
x4=np.sum(np.equal(res3, 3))
x5=np.sum(np.equal(res3, 4))
x6=np.sum(np.equal(res3, 5))
x7=np.sum(np.equal(res3, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res4, 0))
x2=np.sum(np.equal(res4, 1))
x3=np.sum(np.equal(res4, 2))
x4=np.sum(np.equal(res4, 3))
x5=np.sum(np.equal(res4, 4))
x6=np.sum(np.equal(res4, 5))
x7=np.sum(np.equal(res4, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res5, 0))
x2=np.sum(np.equal(res5, 1))
x3=np.sum(np.equal(res5, 2))
x4=np.sum(np.equal(res5, 3))
x5=np.sum(np.equal(res5, 4))
x6=np.sum(np.equal(res5, 5))
x7=np.sum(np.equal(res5, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res6, 0))
x2=np.sum(np.equal(res6, 1))
x3=np.sum(np.equal(res6, 2))
x4=np.sum(np.equal(res6, 3))
x5=np.sum(np.equal(res6, 4))
x6=np.sum(np.equal(res6, 5))
x7=np.sum(np.equal(res6, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res7, 0))
x2=np.sum(np.equal(res7, 1))
x3=np.sum(np.equal(res7, 2))
x4=np.sum(np.equal(res7, 3))
x5=np.sum(np.equal(res7, 4))
x6=np.sum(np.equal(res7, 5))
x7=np.sum(np.equal(res7, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

confusion=np.array(confusion)
print(confusion)

[[ 45   6   1   0   4   0   5]
 [  5 153   2   0   1  10   1]
 [ 11   3   7   0   8   0   4]
 [  1   7   1   0   1   0   0]
 [  5   4   6   0  11   0   1]
 [  0   5   0   0   1  27   1]
 [  0   5   1   0   2   2 199]]


In [42]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data2[0:382,:]
X_test = data2[382:,:]
y_train = label[0:382]
y_test = label[382:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(382, 27)
(164, 27)
(382,)
(164,)


In [43]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(382, 7)
(164, 7)


In [44]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 10

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.80137825
Epoch: 2 cost = 1.36809196
Epoch: 3 cost = 1.14299135
Epoch: 4 cost = 1.00908651
Epoch: 5 cost = 0.89312782
Epoch: 6 cost = 0.79392278
Epoch: 7 cost = 0.71725619
Epoch: 8 cost = 0.65758114
Epoch: 9 cost = 0.60785573
Epoch: 10 cost = 0.56552258
Epoch: 11 cost = 0.53022883
Epoch: 12 cost = 0.50170109
Epoch: 13 cost = 0.47887480
Epoch: 14 cost = 0.46013904
Epoch: 15 cost = 0.44409614
Epoch: 16 cost = 0.42990943
Epoch: 17 cost = 0.41723239
Epoch: 18 cost = 0.40606282
Epoch: 19 cost = 0.39655657
Epoch: 20 cost = 0.38872391
Epoch: 21 cost = 0.38206463
Epoch: 22 cost = 0.37541331
Epoch: 23 cost = 0.36734595
Epoch: 24 cost = 0.35696362
Epoch: 25 cost = 0.34439347
Epoch: 26 cost = 0.33070198
Epoch: 27 cost = 0.31726723
Epoch: 28 cost = 0.30526427
Epoch: 29 cost = 0.29608591
Epoch: 30 cost = 0.29360167
Epoch: 31 cost = 0.31603745
Epoch: 32 cost = 0.39064542
Epoch: 33 cost = 0.36222870
Epoch: 34 cost = 0.24632453
Epoch: 35 cost = 0.25149915
Epoch: 36 cost = 0.21014972
E

Epoch: 321 cost = 0.01646258
Epoch: 322 cost = 0.01459153
Epoch: 323 cost = 0.01498398
Epoch: 324 cost = 0.01215681
Epoch: 325 cost = 0.01343067
Epoch: 326 cost = 0.01362933
Epoch: 327 cost = 0.01195887
Epoch: 328 cost = 0.01179672
Epoch: 329 cost = 0.01203834
Epoch: 330 cost = 0.01122821
Epoch: 331 cost = 0.01225656
Epoch: 332 cost = 0.01226912
Epoch: 333 cost = 0.01195330
Epoch: 334 cost = 0.01124756
Epoch: 335 cost = 0.01100072
Epoch: 336 cost = 0.01203823
Epoch: 337 cost = 0.01096478
Epoch: 338 cost = 0.01136718
Epoch: 339 cost = 0.01220075
Epoch: 340 cost = 0.01115766
Epoch: 341 cost = 0.01003349
Epoch: 342 cost = 0.01184471
Epoch: 343 cost = 0.01055364
Epoch: 344 cost = 0.01114157
Epoch: 345 cost = 0.01150157
Epoch: 346 cost = 0.01043504
Epoch: 347 cost = 0.01045338
Epoch: 348 cost = 0.01212736
Epoch: 349 cost = 0.01127517
Epoch: 350 cost = 0.01258522
Epoch: 351 cost = 0.00988446
Epoch: 352 cost = 0.00982569
Epoch: 353 cost = 0.01099866
Epoch: 354 cost = 0.01093986
Epoch: 355 cos

Epoch: 629 cost = 0.00931184
Epoch: 630 cost = 0.01004010
Epoch: 631 cost = 0.00904688
Epoch: 632 cost = 0.00888310
Epoch: 633 cost = 0.01023190
Epoch: 634 cost = 0.00917176
Epoch: 635 cost = 0.00902181
Epoch: 636 cost = 0.00911123
Epoch: 637 cost = 0.01061349
Epoch: 638 cost = 0.01089167
Epoch: 639 cost = 0.01046215
Epoch: 640 cost = 0.00924512
Epoch: 641 cost = 0.00890704
Epoch: 642 cost = 0.00975204
Epoch: 643 cost = 0.00878145
Epoch: 644 cost = 0.00859699
Epoch: 645 cost = 0.01073611
Epoch: 646 cost = 0.01033029
Epoch: 647 cost = 0.00829806
Epoch: 648 cost = 0.00799986
Epoch: 649 cost = 0.00979385
Epoch: 650 cost = 0.01011673
Epoch: 651 cost = 0.00796425
Epoch: 652 cost = 0.00872170
Epoch: 653 cost = 0.00981222
Epoch: 654 cost = 0.70787688
Epoch: 655 cost = 0.63126547
Epoch: 656 cost = 0.04129596
Epoch: 657 cost = 0.01183865
Epoch: 658 cost = 0.01029649
Epoch: 659 cost = 0.00960724
Epoch: 660 cost = 0.00953246
Epoch: 661 cost = 0.00896892
Epoch: 662 cost = 0.00859450
Epoch: 663 cos

Epoch: 926 cost = 0.00698689
Epoch: 927 cost = 0.00807028
Epoch: 928 cost = 0.00816856
Epoch: 929 cost = 0.00786918
Epoch: 930 cost = 0.00725056
Epoch: 931 cost = 0.00826377
Epoch: 932 cost = 0.00814025
Epoch: 933 cost = 0.00799816
Epoch: 934 cost = 0.00678915
Epoch: 935 cost = 0.00774849
Epoch: 936 cost = 0.00669644
Epoch: 937 cost = 0.00813259
Epoch: 938 cost = 0.00789246
Epoch: 939 cost = 0.00800041
Epoch: 940 cost = 0.00688221
Epoch: 941 cost = 0.00691061
Epoch: 942 cost = 0.00713919
Epoch: 943 cost = 0.00820027
Epoch: 944 cost = 0.00750658
Epoch: 945 cost = 0.00698172
Epoch: 946 cost = 0.00843029
Epoch: 947 cost = 0.00752293
Epoch: 948 cost = 0.00868028
Epoch: 949 cost = 0.00730714
Epoch: 950 cost = 0.00806592
Epoch: 951 cost = 0.00707796
Epoch: 952 cost = 0.00779406
Epoch: 953 cost = 0.00753357
Epoch: 954 cost = 0.00790471
Epoch: 955 cost = 0.00805237
Epoch: 956 cost = 0.00903550
Epoch: 957 cost = 0.00754159
Epoch: 958 cost = 0.00720904
Epoch: 959 cost = 0.00720360
Epoch: 960 cos

In [45]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.7743902
